# Trinkgeld Calculator

In [3]:
import pandas as pd

path = ('./data/sample01.xlsx')
xl = pd.ExcelFile(path)
print(xl.sheet_names)

['Übersicht', 'Emelie Appl', 'Açelya Basili', 'Fabrizio Bellia', 'Anna Sophie Blessing', 'Sabulla Bullach', 'Manuel Bär', 'Markus Fuchs', 'Anna Graf', 'Ekaterina Grashchenko', 'Laura Greidenweiß', 'Svenja Halamek', 'Marie Jaussi', 'Okechulowu Ohaebsim', 'Philine Ostermayer', 'Damiano Parziale', 'Luca Principi co Bischoff', 'Pia Schmolly', 'Mauro Sirigu', 'Matthias Strobel', 'Luca Tegeder', 'Jana Wegenke', 'Lilli Wittig', 'Jennifer Wöhrlin']
